In [1]:
import pandas as pd
df = pd.read_csv("final_translated.csv")
df.columns

Index(['Unnamed: 0', 'Gender', 'Hospitalization', 'Death', 'Ventilator',
       'Pneumonia', 'Age', 'Pregnant', 'Diabetes', 'Asthma',
       'Immunosuppression', 'Hypertension', 'Cardiovascular Disease',
       'Obesity ', 'Chronic Renal Insufficiency', 'Tobacco Use',
       'Contact COVID', 'ICU'],
      dtype='object')

In [2]:
df = df.drop(["Hospitalization","ICU","Death"],axis=1)

In [3]:
target = df['Ventilator']

In [4]:
x_values = df.drop(['Ventilator'],axis=1)

In [5]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_values['Age'] = min_max_scaler.fit_transform(x_values[['Age']])
x_values.head()

/home/capeie/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


,Unnamed: 0,Gender,Pneumonia,Age,Pregnant,Diabetes,Asthma,Immunosuppression,Hypertension,Cardiovascular Disease,Obesity,Chronic Renal Insufficiency,Tobacco Use,Contact COVID
0,0,1,1,0.765306,2,1,2,2,2,2,2,2,2,2
1,1,2,2,0.224490,97,2,2,2,2,2,2,2,2,1
2,5,2,1,0.408163,97,2,2,2,2,2,2,2,2,2
3,6,1,2,0.295918,2,2,2,2,2,2,2,2,2,1
4,13,1,2,0.622449,2,1,2,2,2,2,1,2,2,1


In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
column_trans = make_column_transformer((OneHotEncoder(),['Gender','Pneumonia','Pregnant','Diabetes','Asthma','Immunosuppression','Hypertension','Cardiovascular Disease','Obesity ','Chronic Renal Insufficiency','Tobacco Use','Contact COVID']),remainder='passthrough')

In [7]:
y_noob = df.Ventilator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
rfc_pipeline = Pipeline(steps = [
    ('data_preprocessing', column_trans),
    ('model', RandomForestClassifier(max_depth = 10,
                                     min_samples_leaf = 3,
                                     min_samples_split = 4,
                                     n_estimators = 200))
])

In [8]:
#train,test,val split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_values, y_noob, random_state = 42)

In [9]:
rfc_pipeline.fit(x_train, y_train)

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Gender', 'Pneumonia',
                                                   'Pregnant', 'Diabetes',
                                                   'Asthma',
                                                   'Immunosuppression',
                                                   'Hypertension',
                                                   'Cardiovascular Disease',
                                                   'Obesity ',
                                                   'Chronic Renal '
                                                   'Insufficiency',
                                                   'Tobacco Use',
                                                   'Contact COVID'])])),
   

In [10]:
import joblib
joblib.dump(rfc_pipeline, 'models/rfc_pipeline_Ventilator.pkl')

['models/rfc_pipeline_Ventilator.pkl']

In [17]:
loaded_model = joblib.load('models/rfc_pipeline_Ventilator.pkl')
pipeline_preds = loaded_model.predict(x_val)
cross_val_score(rfc_pipeline,x_val,y_val,cv=5,scoring='accuracy').mean()


0.802332974893637

In [18]:
x_new = x_values.sample(1,random_state=99)
pipeline_preds = loaded_model.predict(x_new)

In [19]:
x_new

,Unnamed: 0,Gender,Pneumonia,Age,Pregnant,Diabetes,Asthma,Immunosuppression,Hypertension,Cardiovascular Disease,Obesity,Chronic Renal Insufficiency,Tobacco Use,Contact COVID
115,742,1,1,0.581633,2,2,2,2,2,2,2,2,2,2


In [20]:
prediction_translate = {1:'Needs Ventilator',2:'No Ventilator required',97:'Not Applicable',98:'Ignored',99:'Not Specified'}

In [21]:
prediction_translate[pipeline_preds[0]]

'No Ventilator required'